In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
import requests
import numpy as np

/Users/apple/opt/anaconda3/envs/chatbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/apple/opt/anaconda3/envs/chatbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/apple/opt/anaconda3/envs/chatbot/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/apple/opt/anaconda3/envs

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
def get_vect(query_in, lang = 'en', address = '127.0.0.1:8050'):
    url = "http://" + address + "/vectorize"
    params = {"q": query_in, "lang": lang}
    resp = requests.get(url=url, params=params).json()
    return resp["embedding"]

In [4]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
from googletrans import Translator
translator = Translator()

In [7]:
with open("banking_intents.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    training = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            temp = get_vect(pattern)[0]
            print(pattern)
            training.append(temp)
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
            # data augmentation using google translator for improving the accuracy - start
            # Prediction model can work even without this.
            translation = translator.translate(pattern, dest='hi').text
            temp = get_vect(translation)[0]
            print(translation)
            training.append(temp)
            wrds = nltk.word_tokenize(translation)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            # data augmentation using google translator for improving the accuracy - end
            
            transliteration = transliterate(translation, sanscript.DEVANAGARI, sanscript.ITRANS)
            temp = get_vect(transliteration)[0]
            print(transliteration)
            training.append(temp)
            wrds = nltk.word_tokenize(transliteration)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
            transliteration_hi = transliterate(transliteration, sanscript.ITRANS, sanscript.DEVANAGARI)
            temp = get_vect(transliteration_hi)[0]
            print(transliteration_hi)
            training.append(temp)
            wrds = nltk.word_tokenize(transliteration_hi)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    output = []
    #print(training)
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

how can i apply for personal loan
मैं व्यक्तिगत ऋण के लिए कैसे आवेदन कर सकता हूं
maiM vyaktigata RRiNa ke lie kaise Avedana kara sakatA hUM
I want to apply for personal loan
मैं पर्सनल लोन के लिए आवेदन करना चाहता हूं
maiM parsanala lona ke lie Avedana karanA chAhatA hUM
I would like to apply for personal loan
मैं पर्सनल लोन के लिए आवेदन करना चाहूंगा
maiM parsanala lona ke lie Avedana karanA chAhUMgA
where can i apply for personal loan
मैं व्यक्तिगत ऋण के लिए कहां आवेदन कर सकता हूं
maiM vyaktigata RRiNa ke lie kahAM Avedana kara sakatA hUM
application for personal loan
व्यक्तिगत ऋण के लिए आवेदन
vyaktigata RRiNa ke lie Avedana
personal loan application
व्यक्तिगत ऋण आवेदन
vyaktigata RRiNa Avedana
how can i apply for business loan
मैं व्यवसाय ऋण के लिए कैसे आवेदन कर सकता हूं
maiM vyavasAya RRiNa ke lie kaise Avedana kara sakatA hUM
I want to apply for business loan
मैं बिजनेस लोन के लिए आवेदन करना चाहता हूं
maiM bijanesa lona ke lie Avedana karanA chAhatA hUM
I would like to apply for busi

maiM eka bachata khAtA kholanA chAhUMgA
open savings account
बचत खाता खोलें
bachata khAtA kholeM
savings account opening procedure
बचत खाता खोलने की प्रक्रिया
bachata khAtA kholane kI prakriyA
savings account open
बचत खाता खुला
bachata khAtA khulA
how much interest do i get in a current account
मुझे चालू खाते में कितना ब्याज मिलता है
mujhe chAlU khAte meM kitanA byAja milatA hai
what is the interest rate for current account
चालू खाते के लिए ब्याज दर क्या है
chAlU khAte ke lie byAja dara kyA hai
current account interest rate
चालू खाता ब्याज दर
chAlU khAtA byAja dara
interest rate for current account
चालू खाते के लिए ब्याज दर
chAlU khAte ke lie byAja dara
how much interest do i get in a savings account
बचत खाते में मुझे कितना ब्याज मिलता है
bachata khAte meM mujhe kitanA byAja milatA hai
what is the interest rate for savings account
बचत खाते के लिए ब्याज दर क्या है
bachata khAte ke lie byAja dara kyA hai
savings account interest rate
बचत खाता ब्याज दर
bachata khAtA byAja dara
interest ra

In [8]:
try:
    model.load("model.tflearn")
except:
    net = tflearn.input_data(shape=[None, len(training[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
    net = tflearn.regression(net)

    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

Training Step: 31999  | total loss: 0.00004 | time: 0.074s
| Adam | epoch: 1000 | loss: 0.00004 - acc: 1.0000 -- iter: 248/249
Training Step: 32000  | total loss: 0.00004 | time: 0.077s
| Adam | epoch: 1000 | loss: 0.00004 - acc: 1.0000 -- iter: 249/249
--
INFO:tensorflow:/Users/apple/Documents/Chatbot Challenge/dnn/Classifier5/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [22]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([get_vect(inp)[0]])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        print(results[0][results_index])
        if results[0][results_index] > 0.9 :
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
        else:
            print("Please try again")
        

In [23]:
chat()

Start talking with the bot (type quit to stop)!
You: dsn kqelqav n ad
0.98616713
Your appointment is scheduled
You: i want to watch a movie
0.6812361
Please try again
You: mmujhe personal loan apply karna chahye
0.99865663
Please click here to know the procedure to apply for credit card
You: mujhe personal loan apply karna chahiye
0.9995803
Please click here to know the procedure to apply for credit card
You: मैं डेबिट कार्ड कैसे जोड़ सकता हूं
0.9999815
Please click here to know the procedure for debit card addon
You: quit


In [11]:
def get_lang(utterance):
    url = "http://127.0.0.1:5000/detect_lang"
    params = {"utterance": utterance}
    resp = requests.get(url=url, params=params)
    if resp.text == "1":
        return "hi"
    else:
        return "en"

In [32]:
from polyglot.detect import Detector
def detect_lang(utterance):
    detector = Detector(utterance,quiet=True)
    return detector.language.code

In [33]:
import json
import requests
from urllib.parse import  quote

def print_entities(utterance, lang):
    text=quote(utterance)
    url='http://localhost:8081/v2/date/?entity_name=date&timezone=UTC&past_date_referenced=false&unit_type=&source_language={lang}&message={utterance}'.format(utterance=text, lang=lang)
    response = requests.get(url)
    response.json()
    data = []
    if(response.json()['data']):
        for el in response.json()['data']:
            print(el['entity_value']['value'])
            data.append(el)
    
    url='http://localhost:8081/v2/number/?entity_name=number_of_unit&min_number_digits=1&max_number_digits=10&unit_type=&message={}'.format(text)
    response = requests.get(url)
    response.json()
    if(response.json()['data']):
        for el in response.json()['data']:
            print(el['entity_value']['value'])
            data.append(el)
    
    return data

In [46]:
def chat_latest():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        lang = "en"
        if detect_lang(inp) == "hi":
            lang = "hi"
            print("detected language : {}".format("hi"))
        else:
            if get_lang(inp) == "hi":
                lang = "hi"
                print("detected language : {}".format("hi - transliterate"))
                inp = transliterate(inp, sanscript.ITRANS, sanscript.DEVANAGARI)
                print("transliterate to hindi as : {}".format(inp))
            else:
                print("detected language : {}".format("en"))
                
        print_entities(inp, lang)
        
        results = model.predict([get_vect(inp)[0]])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

In [47]:
chat_latest()

Start talking with the bot (type quit to stop)!
You: i need 10 lakh rupees at 5% tomorrow
0
detected language : en
{'dd': 7, 'mm': 11, 'yy': 2020, 'type': 'tomorrow'}
1000000
5
Please click here to know the procedure to apply for credit card
You: chAlU khAte ke lie byAja dara kyA hai agla somavar


Detector is not able to detect the language reliably.


1
detected language : hi - transliterate
transliterate to hindi as : चालू खाते के लिए ब्याज दर क्या है अग्ल सोमवर्
Please click here to apply for business loan
You: मैं व्यक्तिगत ऋण के लिए कैसे आवेदन कर सकता हूं
detected language : hi
Please click here to apply for personal loan
You: मैं व्यक्तिगत ऋण के लिए आवेदन करना चाहता हूं
detected language : hi
Please click here to apply for personal loan
You: मैं personal loan के लिए आवेदन करना चाहता हूं
detected language : hi
Please click here to apply for personal loan
You: व्यापार ऋण के लिए आवेदन करना
detected language : hi
Please click here to apply for business loan
You: business loan application
0
detected language : en
Please click here to apply for business loan
You: vyaapar run ke liye aavedhan karna
1
detected language : hi - transliterate
transliterate to hindi as : व्यापर् रुन् के लिये आवेधन् कर्न
Please click here to know the procedure to apply for credit card
You: व्यापर् रुन् के लिये आवेधन् कर्न
detected language : hi
Please click

Detector is not able to detect the language reliably.


1
detected language : hi - transliterate
transliterate to hindi as : व्यापर् रुन् के लिये आवेधन् कर्ना
Please click here to know the procedure to apply for credit card
You: vyaapaar run ke liye aavedhan karnaa


...

detected language : hi
Please click here to apply for personal loan
You: personal loan के लिए apply करना


Detector is not able to detect the language reliably.


detected language : hi
Please click here to apply for personal loan
You: कृपया क्रेडिट कार्ड आवेदन विवरणों के साथ मदद करें
detected language : hi
Please click here to know the procedure to apply for credit card
You: अधिकतम व्यक्तिगत ऋण राशि क्या है
detected language : hi
Please click here to know max and min amount for personal loan
You: व्यक्तिगत ऋण राशि क्या है
detected language : hi
Please click here to apply for personal loan
You: मैं एक नियुक्ति चाहता हूं
detected language : hi
Your appointment is scheduled
You: मैं 10 lakh व्यक्तिगत ऋण के लिए कैसे आवेदन कर सकता हूं
detected language : hi
1000000
Please click here to apply for personal loan
You: मैं अगले महाधिवेशन की नियुक्ति करना चाहता हूं
detected language : hi
Your appointment is scheduled
You: मैं अगले सोमवार की नियुक्ति करना चाहता हूं
detected language : hi
{'dd': 16, 'mm': 11, 'yy': 2020, 'type': 'date'}
Your appointment is scheduled
You: मैं एक current account खोलना चाहता हूं
detected language : hi
Please click here to know

Detector is not able to detect the language reliably.


0
detected language : en
Please click here to apply for business loan
You: मैं डेबिट कार्ड के लिए आवेदन करना चाहूंगा
detected language : hi
Please click here to know the procedure to apply for debit card
You: अधिकतम व्यक्तिगत ऋण राशि क्या है
detected language : hi
Please click here to know max and min amount for personal loan
You: डेबिट कार्ड एडऑन
detected language : hi
Please click here to know the procedure for debit card addon
You: मैं एक चालू खाता खोलना चाहता हूं
detected language : hi
Your appointment is scheduled
You: व्यक्तिगत ऋण राशि क्या है
detected language : hi
Please click here to apply for personal loan
You: current account खुला
1
detected language : hi - transliterate
transliterate to hindi as : cउर्रेन्त् अccओउन्त् खुला
Please click here to know the procedure to apply for credit card
You: मुझे current account में कितना ब्याज मिलता है
detected language : hi
Your appointment is scheduled
You: current account के लिए ब्याज दर क्या है
detected language : hi
Your appointment i

In [55]:
chat_latest()

Start talking with the bot (type quit to stop)!
You: mujhe current account open karna
0
detected language : en
Your appointment is scheduled
You: mujhe current account open chahiye
0
detected language : en
Your appointment is scheduled
You: quit


In [37]:
def chat_api(inp):
    lang = "en"
    response = {}
    if detect_lang(inp) == "hi":
        lang = "hi"
        print("detected language : {}".format("hi"))
        response['lang'] = 'hi'
    else:
        if get_lang(inp) == "hi":
            lang = "hi"
            print("detected language : {}".format("hi - transliterate"))
            response['lang'] = 'hi - transliterate'
            inp = transliterate(inp, sanscript.ITRANS, sanscript.DEVANAGARI)
            print("transliterate to hindi as : {}".format(inp))
        else:
            response['lang'] = 'en'
            print("detected language : {}".format("en"))

    response['entities'] = print_entities(inp, lang)

    results = model.predict([get_vect(inp)[0]])
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    if results[0][results_index] > 0.9 :
        response['intent'] = tag 
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
    else:
         response['intent'] = 'unknown'
    
    return jsonify(response)
    print(random.choice(responses))

In [ ]:
from flask import Response, Flask, jsonify, request
app = Flask(__name__)

@app.route('/predict', methods=['GET'])
def predict():
    utterance = request.args.get('utterance')
    return chat_api(utterance)
    
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 7000, app)

 * Running on http://localhost:7000/ (Press CTRL+C to quit)
Detector is not able to detect the language reliably.


detected language : hi - transliterate
transliterate to hindi as : हि


127.0.0.1 - - [27/Nov/2020 06:50:24] "GET /predict?utterance=hi HTTP/1.1" 200 -


detected language : en
1000000


127.0.0.1 - - [27/Nov/2020 06:50:47] "GET /predict?utterance=i%20want%20to%20apply%20for%2010%20lakh%20personal%20loan HTTP/1.1" 200 -
